<a href="https://colab.research.google.com/github/Aravindh4404/FYPSeagullClassification01/blob/main/Refinement_LatestVGG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
import cv2
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from torchvision import transforms
from pathlib import Path
from tqdm import tqdm
from datetime import datetime
import seaborn as sns

# -------------------------------------------------------------------------
# 1) Define Your Modified VGG Model
# (Make sure the parameters match what you used during training)
# -------------------------------------------------------------------------
class VGG16Modified(nn.Module):
    def __init__(self, dropout_rate=0.5, hidden_units=[512]):
        super(VGG16Modified, self).__init__()
        # Load the pretrained VGG16 model
        self.vgg = models.vgg16(weights=models.VGG16_Weights.IMAGENET1K_V1)
        num_ftrs = self.vgg.classifier[6].in_features

        # Build your custom classifier
        classifier_layers = []
        for units in hidden_units:
            classifier_layers.extend([
                nn.Linear(num_ftrs, units),
                nn.ReLU(),
                nn.Dropout(dropout_rate)
            ])
            num_ftrs = units
        classifier_layers.append(nn.Linear(num_ftrs, 2))  # Binary classification

        # Replace the original classifier's final layer
        self.vgg.classifier[6] = nn.Sequential(*classifier_layers)

    def forward(self, x):
        return self.vgg(x)

# -------------------------------------------------------------------------
# 2) Load the Trained Checkpoint
# -------------------------------------------------------------------------
checkpoint_path = "/content/drive/My Drive/FYP/VGGModel/HQ3Optuna_20250218/latest_checkpoint.pth"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = VGG16Modified(dropout_rate=0.5, hidden_units=[512]).to(device)
model.eval()

if not os.path.exists(checkpoint_path):
    print(f"ERROR: The checkpoint file was not found at:\n  {checkpoint_path}\nPlease check the path or filename and try again.")
    exit()
else:
  checkpoint = torch.load(checkpoint_path, map_location=device)
  model.load_state_dict(checkpoint['model_state_dict'])
  print(f"Successfully loaded checkpoint from {checkpoint_path}")
# -------------------------------------------------------------------------
# 3) Define Class Names and Transformations
# -------------------------------------------------------------------------
# (Ensure that your folder names match these names exactly)
class_names = ['Glaucous_Winged_Gull', 'Slaty_Backed_Gull']

transform = transforms.Compose([
    transforms.Resize((224, 224)),  # VGG expects 224x224 input
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

def preprocess_image(image_path):
    """
    Load an image and apply the necessary transforms.
    Returns a tensor of shape (1, 3, 224, 224) on the chosen device.
    """
    image = Image.open(image_path).convert("RGB")
    return transform(image).unsqueeze(0).to(device)

# -------------------------------------------------------------------------
# 4) Grad-CAM Implementation for VGG
# -------------------------------------------------------------------------
def generate_gradcam(model, image_tensor, target_layer):
    """
    Runs a forward and backward pass on the image_tensor, computes the gradients
    on the specified target_layer, and returns the normalized Grad-CAM heatmap,
    predicted class index, and prediction confidence.
    """
    model.eval()
    features = []
    grads = []

    def forward_hook(module, input, output):
        features.append(output)

    def backward_hook(module, grad_in, grad_out):
        grads.append(grad_out[0])

    # Register hooks on the target layer
    forward_handle = target_layer.register_forward_hook(forward_hook)
    backward_handle = target_layer.register_backward_hook(backward_hook)

    # Forward pass
    outputs = model(image_tensor)
    probs = F.softmax(outputs, dim=1)
    predicted_class_idx = torch.argmax(probs, dim=1).item()
    predicted_confidence = probs[0, predicted_class_idx].item()

    # Backward pass: compute gradient for the predicted class
    model.zero_grad()
    score = outputs[0, predicted_class_idx]
    score.backward()

    # Extract gradients and feature maps (remove hooks afterwards)
    gradient = grads[0].detach().cpu().numpy()[0]  # shape: (C, H, W)
    feature_map = features[0].detach().cpu().numpy()[0]  # shape: (C, H, W)
    forward_handle.remove()
    backward_handle.remove()

    # Compute the weights and the weighted combination of feature maps
    weights = np.mean(gradient, axis=(1, 2))
    cam = np.zeros(feature_map.shape[1:], dtype=np.float32)
    for i, w in enumerate(weights):
        cam += w * feature_map[i, :, :]

    # Apply ReLU and normalize the CAM
    cam = np.maximum(cam, 0)
    cam = cv2.resize(cam, (image_tensor.shape[3], image_tensor.shape[2]))
    cam = (cam - np.min(cam)) / (np.max(cam) - np.min(cam) + 1e-8)

    return cam, predicted_class_idx, predicted_confidence

# -------------------------------------------------------------------------
# 5) Utility Functions to Save Visualizations & Statistics
# -------------------------------------------------------------------------
def save_visualization(image_path, cam, predicted_class, confidence, true_class, save_path):
    """
    Creates an overlay of the Grad-CAM heatmap on the original image with text annotations
    (including true class, predicted class, correctness, and confidence) and saves it.
    """
    original_img = Image.open(image_path).resize((224, 224), Image.LANCZOS)
    original_np = np.array(original_img)

    # Create heatmap from the CAM
    heatmap = cv2.applyColorMap(np.uint8(255 * cam), cv2.COLORMAP_JET)
    heatmap = cv2.cvtColor(heatmap, cv2.COLOR_BGR2RGB)

    # Blend heatmap with original image
    overlay = np.clip(0.5 * original_np + 0.5 * heatmap, 0, 255).astype(np.uint8)

    # Prepare text annotations
    pred_label = class_names[predicted_class]
    correct = (pred_label == true_class)
    confidence_text = f"{confidence*100:.2f}%"
    result_text = f"True: {true_class} | Pred: {pred_label} | Conf: {confidence_text} | {'Correct' if correct else 'Wrong'}"

    # Add text to the image using OpenCV (convert to BGR for cv2.putText)
    overlay_bgr = cv2.cvtColor(overlay, cv2.COLOR_RGB2BGR)
    cv2.putText(
        overlay_bgr,
        result_text,
        (10, 25),
        cv2.FONT_HERSHEY_SIMPLEX,
        0.6,
        (0, 255, 0) if correct else (0, 0, 255),
        2,
        cv2.LINE_AA
    )
    overlay_rgb = cv2.cvtColor(overlay_bgr, cv2.COLOR_BGR2RGB)
    cv2.imwrite(save_path, cv2.cvtColor(overlay_rgb, cv2.COLOR_RGB2BGR))

def save_statistics_csv(stat_list, csv_path):
    """
    Save a list of dictionaries (statistics for each image) into a CSV file.
    """
    df = pd.DataFrame(stat_list)
    df.to_csv(csv_path, index=False)
    print(f"Statistics saved to {csv_path}")

# -------------------------------------------------------------------------
# 6) Process the Dataset Folder and Generate Grad-CAM Visualizations
# -------------------------------------------------------------------------
def process_dataset(dataset_path, output_base, delete_wrong=True):
    """
    Processes images stored in subfolders (each subfolder name is taken as the true class),
    generates Grad-CAM visualizations, saves the output images (organized into 'correct' and
    'wrong' folders per class), logs prediction statistics, generates a confusion matrix,
    and optionally deletes wrongly predicted images.
    """
    dataset_path = Path(dataset_path)
    if not dataset_path.exists():
        print(f"ERROR: Dataset path not found: {dataset_path}")
        return

    # Create an output directory with a timestamp
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    output_dir = Path(output_base) / timestamp
    output_dir.mkdir(parents=True, exist_ok=True)
    print(f"Results will be saved to: {output_dir}")

    stats_list = []

    # Process each subfolder (each corresponding to a true class)
    for subfolder in sorted(dataset_path.iterdir()):
        if not subfolder.is_dir():
            continue

        true_class = subfolder.name
        if true_class not in class_names:
            print(f"Skipping folder '{true_class}' as it is not in class_names")
            continue

        # Create output subdirectories for correct and wrong predictions
        out_class_dir = output_dir / true_class
        correct_dir = out_class_dir / "correct"
        wrong_dir = out_class_dir / "wrong"
        correct_dir.mkdir(parents=True, exist_ok=True)
        wrong_dir.mkdir(parents=True, exist_ok=True)

        # Process image files (supports jpg, jpeg, png)
        image_files = list(subfolder.glob("*.jpg")) + list(subfolder.glob("*.jpeg")) + list(subfolder.glob("*.png"))
        print(f"Processing {len(image_files)} images in folder '{true_class}'...")

        # Use the last convolutional layer from VGG for Grad-CAM
        target_layer = model.vgg.features[-1]

        for image_path in tqdm(image_files, desc=f"Processing {true_class}"):
            try:
                # Preprocess image and generate Grad-CAM
                image_tensor = preprocess_image(image_path)
                cam, predicted_class_idx, predicted_confidence = generate_gradcam(model, image_tensor, target_layer)
                predicted_label = class_names[predicted_class_idx]
                is_correct = (predicted_label == true_class)

                # Choose the save directory based on whether the prediction is correct
                save_subdir = correct_dir if is_correct else wrong_dir
                save_filename = f"{image_path.stem}_gradcam.png"
                save_path = str(save_subdir / save_filename)

                # Save the visualization
                save_visualization(str(image_path), cam, predicted_class_idx, predicted_confidence, true_class, save_path)

                # Log the statistics for this image
                stats_list.append({
                    "file": str(image_path),
                    "true_class": true_class,
                    "predicted_class": predicted_label,
                    "confidence": predicted_confidence,
                    "correct": is_correct,
                    "output_file": save_path
                })

                # If the prediction is wrong and deletion is enabled, remove the original image
                if (not is_correct) and delete_wrong:
                    os.remove(str(image_path))

            except Exception as e:
                print(f"Error processing {image_path}: {e}")

    # Save statistics CSV file
    csv_path = output_dir / "statistics.csv"
    save_statistics_csv(stats_list, csv_path)

    # Generate and save the confusion matrix
    try:
        df = pd.DataFrame(stats_list)
        if not df.empty:
            confusion = pd.crosstab(df['true_class'], df['predicted_class'], rownames=['True'], colnames=['Predicted'])
            plt.figure(figsize=(6, 4))
            sns.heatmap(confusion, annot=True, fmt="d", cmap="Blues")
            plt.title("Confusion Matrix")
            cm_path = output_dir / "confusion_matrix.png"
            plt.savefig(cm_path, dpi=300, bbox_inches='tight')
            plt.close()
            print(f"Confusion matrix saved to: {cm_path}")
    except Exception as e:
        print(f"Error generating confusion matrix: {e}")

# -------------------------------------------------------------------------
# 7) Main Execution: Set Your Dataset and Output Folders
# -------------------------------------------------------------------------
if __name__ == "__main__":
    # Specify the parent folder containing two class subfolders
    dataset_path = "/content/drive/My Drive/FYP/Black Background/"

    # Specify the base output directory (can be on Google Drive or local)
    output_base = "/content/drive/My Drive/FYP/VGGAnalysis"

    # Set delete_wrong=True to permanently remove wrongly predicted images from the source folder
    process_dataset(dataset_path, output_base, delete_wrong=True)


<ipython-input-2-7a80aafdbbc0>:57: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_path, map_location=device)


Successfully loaded checkpoint from /content/drive/My Drive/FYP/VGGModel/HQ3Optuna_20250218/latest_checkpoint.pth
Results will be saved to: /content/drive/My Drive/FYP/VGGAnalysis/20250218_123021
Processing 117 images in folder 'Glaucous_Winged_Gull'...


Processing Glaucous_Winged_Gull: 100%|██████████| 117/117 [04:28<00:00,  2.30s/it]


Processing 142 images in folder 'Slaty_Backed_Gull'...


Processing Slaty_Backed_Gull: 100%|██████████| 142/142 [06:40<00:00,  2.82s/it]


Statistics saved to /content/drive/My Drive/FYP/VGGAnalysis/20250218_123021/statistics.csv
Confusion matrix saved to: /content/drive/My Drive/FYP/VGGAnalysis/20250218_123021/confusion_matrix.png
